# Creator Metrics

This script processes a CSV from [this Dune Query](https://dune.com/queries/4087193) and derives metrics the following NFT creator metrics:

- num drops
- num unique minters
- num txns
- USD value of txns

In [1]:
from dotenv import load_dotenv
from dune_client.types import QueryParameter
from dune_client.client import DuneClient
from dune_client.query import QueryBase
import json
import os
import pandas as pd

2024-10-04 12:04:06,400 INFO numexpr.utils NumExpr defaulting to 8 threads.


In [2]:
load_dotenv()

DUNE_API_KEY = os.getenv('DUNE_API_KEY')
dune = DuneClient(DUNE_API_KEY)

## Part 1. Parse the creator data

In [3]:
apps = pd.read_csv('data/apps/creator_apps.csv', index_col=0)
apps = apps.drop(columns=['chain']).drop_duplicates()
uuid_to_address_mapper = apps.set_index('uuid')['address'].to_dict()
uuid_list = list(apps['uuid'].unique())
address_list = list(apps['address'].unique())
apps.tail(1)

,uuid,recipient,project_type,category,address
1518,568706f3-8127-448f-b0b1-56c28e4f3ed1,0xf56E55e35d2CCa5A34F5Ba568454974424aEA0F4,Creator,Art NFTs,0xf56e55e35d2cca5a34f5ba568454974424aea0f4


In [4]:
all_apps = pd.read_csv('data/apps/applications_reviewed.csv')
all_apps = all_apps[all_apps['uuid'].isin(uuid_list)]
uuid_to_profile_mapper = all_apps.set_index('uuid')['profile_name'].to_dict()
uuid_to_name_mapper = all_apps.set_index('uuid')['name'].to_dict()
uuid_to_id_mapper = all_apps.set_index('uuid')['id'].to_dict()
all_apps.tail(1)

,uuid,charmverseId,agoraProjectRefUID,id,recipient,time,name,status,profile_name,profile_url,...,chain_id,chain,flag_multiple_projects_same_profile,flag_creator_no_address,flag_app_missing_contract,flag_channel_no_channel,flag_charmverse_in_name,flag_creator_address_conflict,count_flags,has_flag
1613,568706f3-8127-448f-b0b1-56c28e4f3ed1,568706f3-8127-448f-b0b1-56c28e4f3ed1,0x2b24be9c82b7cbd266af7a2a27fbd3213c98bd93c01d...,0xb07fdf1ede14ea0abf845c015d4ad73f336e50acb273...,0xf56E55e35d2CCa5A34F5Ba568454974424aEA0F4,1725398873,面白い事実,pending,music-guy.eth,https://warpcast.com/music-guy.eth,...,NaN,All Superchain,False,False,False,False,False,False,0,0


In [5]:
#all_apps[all_apps['name'].str.contains('Kismet')].T
len(all_apps)

507

In [6]:
app_dict = {}
for _,app in apps.iterrows():
    recipient = app['recipient'].lower()
    uuid = app['uuid']
    minting_address = app['address'].lower()
    if recipient not in app_dict:
        app_dict.update({
            recipient: {
                'mintingAddressList': [minting_address],
                'uuidList': [uuid]
            }
        })
    else:
        app_dict[recipient]['mintingAddressList'].append(minting_address)
        app_dict[recipient]['uuidList'].append(uuid)

app_json = []        
for recipient, record in app_dict.items():
    app_json.append({
        'recipientAddress': recipient,
        'mintingAddressList': list(set(record['mintingAddressList'])),
        'uuidList': list(set(record['uuidList'])),
    })
        
with open("data/apps/creator_address_to_uuid.json", "w") as f:
    json.dump(app_json, f, indent=2)

In [7]:
farcaster_df = pd.read_parquet('data/raw_metric_data/farcaster.parquet')
fids = farcaster_df.set_index('address')['fid'].to_dict()

## Part 2. Grab NFT mint data from Dune

In [8]:
creator_addresses_str = ',\n\t\t'.join(address_list)
query_sql = f"""

with nft_creators as (
    select
        nft_contract_address,
        blockchain,
        min_by(tx_from, block_number) as creator_address
    from nft.mints
    where
        block_date between date('2024-01-01') and date('2024-09-01')
        and blockchain in ('base', 'optimism', 'zora')
    group by 1,2
)

select
    nft_creators.creator_address,
    nft_creators.nft_contract_address,
    nft_creators.blockchain,
    count(distinct case when block_date > date('2024-06-01') then m.tx_hash end) as num_txns_90D,
    count(distinct m.tx_hash) as num_txns_180D,
    sum(case when block_date > date('2024-06-01') then coalesce(m.amount_usd,0) else 0 end) as amount_usd_90D,
    sum(coalesce(m.amount_usd,0)) as amount_usd_180D,
    array_agg(distinct case when block_date > date('2024-06-01') then m.buyer end) as buyers_90D,
    array_agg(distinct m.buyer) as buyers_180D
from nft.mints as m
join nft_creators
    on m.nft_contract_address = nft_creators.nft_contract_address
    and m.blockchain = nft_creators.blockchain
where
    m.block_date between date('2024-02-01') and date('2024-09-01')
    and nft_creators.creator_address in (
                {creator_addresses_str}
    )
group by 1,2,3

"""

In [9]:
query_id = dune.create_query(name='sunny_nft_mints', query_sql=query_sql, is_private=False)
print(query_id.base.query_id)

# query = QueryBase(name='sunny_nft_mints', query_id=query_id.base.query_id)
# results_df = dune.run_query_dataframe(query)
# results_df.to_parquet('data/raw_metric_data/dune_raw_nft_mints.parquet')

nft_creators = pd.read_parquet('data/raw_metric_data/dune_raw_nft_mints.parquet')

nft_creators['blockchain'] = nft_creators['blockchain'].apply(lambda x: x.title())

def clean_lst(lst_str):
    lst_str = lst_str.replace("[","").replace("]","")
    lst = lst_str.split(" ")
    lst = [x for x in lst if x[:2] == '0x']
    return lst

nft_creators['buyers_90D'] = nft_creators['buyers_90D'].apply(clean_lst)
nft_creators['buyers_180D'] = nft_creators['buyers_180D'].apply(clean_lst)

def map_fids(lst_addr):
    return [x for x in set([fids.get(x) for x in lst_addr]) if x is not None]

nft_creators['list_of_fids_90D'] = nft_creators['buyers_90D'].apply(map_fids)
nft_creators['list_of_fids_180D'] = nft_creators['buyers_180D'].apply(map_fids)

nft_creators.tail(1).T

4127920


,2292
creator_address,0x8feee00f1d8c355086b4667f153c744688b7f6b0
nft_contract_address,0x58b84ed3bad4e1e7ae0f29b671b41c689c851330
blockchain,Base
num_txns_90D,0
num_txns_180D,4
amount_usd_90D,0.0
amount_usd_180D,5.216615
buyers_90D,[]
buyers_180D,"[0x8feee00f1d8c355086b4667f153c744688b7f6b0, 0..."
list_of_fids_90D,[]


# Part 3. Derive the metrics

In [10]:
metrics_by_address = pd.concat([
    nft_creators.groupby('creator_address')['nft_contract_address'].nunique().rename('num_drops'),

    nft_creators.groupby('creator_address')['buyers_90D'].agg(lambda x: len(set().union(*x))).rename('num_unique_minters_90D'),
    nft_creators.groupby('creator_address')['buyers_180D'].agg(lambda x: len(set().union(*x))).rename('num_unique_minters_180D'),

    nft_creators.groupby('creator_address')['list_of_fids_90D'].agg(lambda x: len(set().union(*x))).rename('num_farcaster_minters_90D'),
    nft_creators.groupby('creator_address')['list_of_fids_180D'].agg(lambda x: len(set().union(*x))).rename('num_farcaster_minters_180D'),

    nft_creators.groupby('creator_address')['num_txns_90D'].sum().rename('num_transactions_90D'),
    nft_creators.groupby('creator_address')['num_txns_180D'].sum().rename('num_transactions_180D'),
        
    nft_creators.groupby('creator_address')['amount_usd_90D'].sum().rename('usd_value_of_transactions_90D'),
    nft_creators.groupby('creator_address')['amount_usd_180D'].sum().rename('usd_value_of_transactions_180D')
], axis=1)
metrics_by_address

,num_drops,num_unique_minters_90D,num_unique_minters_180D,num_farcaster_minters_90D,num_farcaster_minters_180D,num_transactions_90D,num_transactions_180D,usd_value_of_transactions_90D,usd_value_of_transactions_180D
creator_address,,,,,,,,,
0x00409fc839a2ec2e6d12305423d37cd011279c09,4,8,56,1,7,11,66,3198.829802,46257.845985
0x0438053b82cb27004bd80d3ca18b8ef6b23f20cf,3,31,98,26,84,65,198,3.055382,3.055382
0x05a7eb929209f5c1f2f08dd087b64bb1beba99fe,1,5,38,5,29,5,72,0.000000,2.167263
0x05e0ccdee8e642388da4478facc6ac7c496714f6,4,2,10,2,10,3,35,0.000000,0.000000
0x07bd4932d08e5956fab224965a58bf1a441134a1,45,52420,79621,9117,23166,55748,87180,27471.553135,92138.307501
...,...,...,...,...,...,...,...,...,...
0xf8c80ebf80ff0c17bdfbc08a91e64a890aae8e3c,7,2884,2884,1028,1028,3053,3054,3631.832326,3631.832326
0xf977a728f3aa00885b2de0a106f8ae69d17adb08,3,0,8319,0,5228,0,9025,0.000000,80998.001457
0xf981a7a6b3796bf02425fd64fd80093b6fc5b2ac,3,21,35,18,29,11,36,0.280545,51.731309


In [11]:
metrics = []
errors = []
for uuid, app in apps.set_index('uuid').iterrows():
    
    name = uuid_to_name_mapper.get(uuid)
    address = app['address']
    recipient = app['recipient']
    
    if address not in metrics_by_address.index:
        errors.append({
            'uuid': uuid,
            **app,
            'status': 'no_nft_activity_90D'
        })
        continue
    
    creator_metrics = metrics_by_address.loc[address].to_dict()
    
    metrics.append({
        'uuid': uuid,
        **app,
        'status': 'metrics_available',
        **creator_metrics,
    })

In [12]:
df_metrics = pd.DataFrame(metrics)
df_metrics.to_csv('data/clean_metric_data/metrics_creators.csv')

## Part 4. Derive metrics by recipient address

In [13]:
recipient_mapping = (
    all_apps
    .groupby(['recipient', 'profile_name'])
    ['address']
    .unique()
    .apply(list)
    .to_dict()
)

In [14]:
recipient_metrics = []
for (recipient,profile_name),address_list in recipient_mapping.items():

    dff = nft_creators[nft_creators.creator_address.isin(address_list)]
    if len(dff) < 1:
        continue

    uuid_list =  all_apps[(all_apps['recipient'] == recipient) & (all_apps['profile_name'] == profile_name)]['uuid'].unique()
    app_id_list = list(set([uuid_to_id_mapper[x] for x in uuid_list]))
    name_list = list(set([uuid_to_name_mapper[x] for x in uuid_list]))
    
    
    nft_creators.groupby('creator_address')['buyers_90D'].agg(lambda x: len(set().union(*x))).rename('num_unique_minters_90D'),
    nft_creators.groupby('creator_address')['buyers_180D'].agg(lambda x: len(set().union(*x))).rename('num_unique_minters_180D'),

    nft_creators.groupby('creator_address')['list_of_fids_90D'].agg(lambda x: len(set().union(*x))).rename('num_farcaster_minters_90D'),
    nft_creators.groupby('creator_address')['list_of_fids_180D'].agg(lambda x: len(set().union(*x))).rename('num_farcaster_minters_180D'),

    nft_creators.groupby('creator_address')['num_txns_90D'].sum().rename('num_transactions_90D'),
    nft_creators.groupby('creator_address')['num_txns_180D'].sum().rename('num_transactions_180D'),
        
    nft_creators.groupby('creator_address')['amount_usd_90D'].sum().rename('usd_value_of_transactions_90D'),
    nft_creators.groupby('creator_address')['amount_usd_180D'].sum().rename('usd_value_of_transactions_180D')

    
    creator_metrics = {
        'recipient': recipient,
        'artist_profile': f'"{profile_name}"',
    
        'num_drops': dff['nft_contract_address'].nunique(),
        
        #'num_unique_minters_90D': len(set(dff['buyers_90D'].sum())),
        'num_unique_minters': len(set(dff['buyers_180D'].sum())),
        
        #'num_farcaster_minters_90D': len(set(dff['list_of_fids_90D'].sum())),
        'num_farcaster_minters': len(set(dff['list_of_fids_180D'].sum())),
        
        #'num_transactions_90D': dff['num_txns_90D'].sum(),
        'num_transactions': dff['num_txns_180D'].sum(),
        
        #'usd_value_of_transactions_90D': dff['amount_usd_90D'].sum(),
        'usd_value_of_transactions': dff['amount_usd_180D'].sum(),
        
        'uuid_list': uuid_list,
        'application_id_list': app_id_list
        #'project_names': name_list
    }
    recipient_metrics.append(creator_metrics)

In [15]:
df_recipient_metrics = pd.DataFrame(recipient_metrics).set_index(['recipient', 'artist_profile'])
df_recipient_metrics.to_csv('data/clean_metric_data/metrics_creators_by_recipient.csv')
df_recipient_metrics.sort_values(by='num_transactions')

,,num_drops,num_unique_minters,num_farcaster_minters,num_transactions,usd_value_of_transactions,uuid_list,application_id_list
recipient,artist_profile,,,,,,,
0x2e63A76A0025BF1D92CECdA73c5Efd342849fd0d,"""snip3""",1,1,1,1,1.790511,[6e6228be-6e91-4b8f-bdf8-54ab3a9f3625],[0x3d66e061c487378b5263c244f941a01ca40baf2947c...
0xF15132421A7Cd4a3528d5b938b53c782Fe7fE57D,"""mayowalawal.eth""",1,1,1,1,0.339187,[2612c31b-b3d0-475a-85dc-650fbbfad308],[0x647b118b702796f479900d3ad4275b3fca7d2d8135f...
0x2Ad69BEE4b9858421Fd62f37A9476595e74ad3e6,"""optimistizu""",1,1,1,1,0.000000,[9b25035d-4916-4326-b1cd-ec847ed8da2f],[0xbbaeda9c59c89fbdf705ce2a73a2484096af5b1d38a...
0x435172C1B669839dE487c8A26BeE3124E2f06D78,"""hanma""",1,1,1,1,0.000000,[c50abe5e-ad79-4707-9beb-9e91edef299b],[0x092a6fc3cce81c8eedd578519e396dae9f7881df31c...
0xc609Cfa28553a2Cd21176b2849F27824090A8aA3,"""skyncrypto""",1,1,1,1,0.000000,[f5e827fd-533d-4099-b4f0-33e36b50eb12],[0xc5e78a8a36ffe814631480bd00584d0c384b80ab1cf...
...,...,...,...,...,...,...,...,...
0x212fC47e707eD36409154ebf8bCfd2aA0E26517B,"""0xhellia""",9,61215,9961,83196,20927.591097,[ccd139de-e7b6-4f9c-8058-b46b9e34bb23],[0x07eef6001515b6c4e36ccd93468779a737a6c2ca77b...
0x07BD4932D08e5956faB224965a58BF1a441134A1,"""eneowaith.eth""",45,79621,23166,87180,92138.307501,[caf3b198-91f2-495e-96fd-67b9cb622e73],[0xc604889111ec702c3f8df0337719f54554cc5f8adce...
0x2Fb0e475F6D495DFdfd9176aF9113f48F7687565,"""nimaleophotos.eth""",23,66493,9776,100614,28734.609541,"[ad90d932-3b19-4399-bfa0-bdd3975dc12f, 5a5c218...",[0xde7349646c03ca8b15c5bd50f0ac87626d00888f497...
